# Dependencies and Project Installation

In [1]:
!pip install torch
!pip install ipython
!git clone -b feat/inference/main https://github.com/josephchay/linguify.git

fatal: destination path 'linguify' already exists and is not an empty directory.


In [3]:
%cd linguify
!pip install -r requirements.txt

/content/linguify


In [4]:
import torch
torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision('high')

import src
from IPython.display import Audio

# Load Models

In [5]:
chat = src.Chat()
chat.load_models()

# Use force_redownload=True if the weights updated.
# chat.load_models(force_redownload=True)

# If you download the weights manually, set source='locals'.
# chat.load_models(source='local', local_path='YOUR LOCAL PATH')

/content/linguify/src/core.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vocos.load_state_dict(torch.load(vocos_ckpt_path))
/content/linguify/src/core.py:81: FutureWa

# Inferencing

## Batch inference

In [6]:
texts = [(
        "Sepsis is a life threatening condition characterized by systemic inflammation "
        "and organ dysfunction triggered by infection. "
        "The Surviving Sepsis Campaign recommends initiating broad spectrum antibiotics "
        "within one hour of suspected sepsis, alongside aggressive fluid resuscitation "
        "with crystalloids. "
        "Vasopressor support, particularly norepinephrine, is indicated "
        "if mean arterial pressure, abbreviated as MAP, remains below 65 millimeters "
        "of mercury despite adequate fluid administration."
    )]

wavs = chat.inference(texts)

 55%|█████▌    | 1133/2048 [00:12<00:10, 91.02it/s]


In [7]:
Audio(wavs[0], rate=24_000, autoplay=True)

## Custom Parameters

In [8]:
params_infer_code = {'prompt':'[speed_5]', 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}

wav = chat.inference(("Acute coronary syndrome, presents with a spectrum "
     "of conditions ranging from unstable angina to Segment T elevation myocardial infarction. "
     "Management involves prompt initiation of dual antiplatelet therapy "
     "using aspirin and an inhibitor, alongside anticoagulation "
     "with heparin or enoxaparin."), \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

 33%|███▎      | 672/2048 [00:32<01:06, 20.78it/s] 


In [9]:
Audio(wav[0], rate=24_000, autoplay=True)

## Random Speaker

In [10]:
rand_spk = chat.sample_random_speaker()
params_infer_code = {'spk_emb' : rand_spk, }

wav = chat.inference(("The patient presented with tachycardia, diaphoresis, and altered mentation, "
     "indicative of possible hypoglycemic crisis. "
     "Immediate intervention included administration of 50 percent dextrose "
     "solution intravenously, followed by continuous glucose monitoring "
     "to stabilize serum glucose levels."), \
    params_refine_text=params_refine_text, params_infer_code=params_infer_code)

 48%|████▊     | 977/2048 [00:10<00:11, 93.69it/s]


In [11]:
Audio(wav[0], rate=24_000, autoplay=True)

## Two Stage Control

In [12]:
text = "So we found that maintaining good health works best with a strong support system; someone to remind you when you're off track, someone to encourage you to stay committed, and someone to join you in healthy habits like exercising or meal prepping."
chat.inference(text, refine_text_only=True)

 21%|██        | 80/384 [00:22<01:25,  3.56it/s]


["so we found that maintaining good health [uv_break] works best with a strong support system access someone to remind you when you're off track, [uv_break] someone to encourage you to stay committed, and someone to join you [uv_break] in healthy habits like exercising or meal prepping."]

In [13]:
text = "So we found that maintaining good health [uv_break] works best with a strong support system; [uv_break] someone to remind you [uv_break] when you're off track, [uv_break] someone to encourage you [uv_break] to stay committed, [uv_break] and someone to join you [uv_break] in healthy habits [uv_break] like exercising or meal prepping."
wav = chat.inference(text, skip_refine_text=True)

 46%|████▌     | 943/2048 [00:10<00:11, 93.52it/s]


# Executing LLM

In [14]:
from src.experimental.llm import LLMApi

client = LLMApi()

Loading model deepseek-ai/deepseek-llm-7b-chat on cuda...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [15]:
user_question = 'What is the recommended initial treatment for suspected sepsis according to the Surviving Sepsis Campaign?'
text = client.call(user_question, prompt_version = 'deepseek')
print(text)
text = client.call(text, prompt_version = 'deepseek_TN')
print(text)

The Surviving Sepsis Campaign recommends early recognition and rapid initiation of appropriate therapy for suspected sepsis. The initial treatment should include a combination of fluids, antibiotics, and vasopressors, depending on the patient's condition. It's essential to monitor the patient's vital signs and response to treatment closely, adjusting the therapy as needed. Early goal-directed therapy can significantly improve outcomes in sepsis patients.
The Surviving Sepsis Campaign suggests early recognition and rapid initiation of appropriate therapy for suspected sepsis. The initial treatment should include a combination of fluids, antibiotics, and vasopressors, depending on the patient's condition. It's crucial to monitor the patient's vital signs and response to treatment closely, adjusting the therapy as needed. Early goal-directed therapy can significantly improve outcomes in sepsis patients.


In [ ]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}

wav = chat.inference(text, params_infer_code=params_infer_code)